In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import os
import pickle
import random
import shutil

import glob2 as glob
import tensorflow as tf
from azureml.core import Experiment, Workspace
from azureml.core.run import Run
from tensorflow.keras import callbacks

sys.path.append(str(Path(os.getcwd()) / 'src'))

from config import CONFIG
from constants import REPO_DIR, NUM_INPUT_CHANNELS

sys.path.append(str(REPO_DIR / 'src/common'))

from model_utils.model_plaincnn import create_cnn
from model_utils.preprocessing import preprocess_depthmap, preprocess_targets
from model_utils.lr_finder import LRFinder

In [ ]:
# Make experiment reproducible
tf.random.set_seed(CONFIG.SPLIT_SEED)
random.seed(CONFIG.SPLIT_SEED)

# Get the current run.
run = Run.get_context()

# Offline run. Download the sample dataset and run locally. Still push results to Azure.
if(run.id.startswith("OfflineRun")):
    print("Running in offline mode...")

    # Access workspace.
    print("Accessing workspace...")
    workspace = Workspace.from_config()
    experiment = Experiment(workspace, "training-junkyard")
    run = experiment.start_logging(outputs=None, snapshot_directory=None)

    # Get dataset.
    print("Accessing dataset...")
    dataset_name = CONFIG.DATASET_NAME_LOCAL  # .DATASET_NAME  # DATASET_NAME_LOCAL
    dataset_path = str(REPO_DIR / "data" / dataset_name)
    if not os.path.exists(dataset_path):
        dataset = workspace.datasets[dataset_name]
        dataset.download(target_path=dataset_path, overwrite=False)

# Online run. Use dataset provided by training notebook.
else:
    print("Running in online mode...")
    experiment = run.experiment
    workspace = experiment.workspace
    dataset_path = run.input_datasets["dataset"]
    
dataset_path = os.path.join(dataset_path, "qrcode")

In [ ]:
# Get the QR-code paths.

print("Dataset path:", dataset_path)
#print(glob.glob(os.path.join(dataset_path, "*"))) # Debug
print("Getting QR-code paths...")
qrcode_paths = glob.glob(os.path.join(dataset_path, "*"))
print("qrcode_paths: ", len(qrcode_paths))
assert len(qrcode_paths) != 0

In [ ]:
dataset_path

In [ ]:
# Shuffle and split into train and validate.
random.shuffle(qrcode_paths)
split_index = int(len(qrcode_paths) * 0.8)
qrcode_paths_training = qrcode_paths[:split_index]

# Show split.
print("Paths for training:")
print("\t" + "\n\t".join(qrcode_paths_training))
print(len(qrcode_paths_training))

assert len(qrcode_paths_training) > 0

def get_depthmap_files(paths):
    pickle_paths = []
    for path in paths:
        pickle_paths.extend(glob.glob(os.path.join(path, "**", "*.p")))
    return pickle_paths


# Get the pointclouds.
print("Getting depthmap paths...")
paths_training = get_depthmap_files(qrcode_paths_training)

print("Using {} files for training.".format(len(paths_training)))

# Function for loading and processing depthmaps.
# def tf_load_pickle(path, max_value):
#     def py_load_pickle(path, max_value):
#         rgbd, targets = pickle.load(open(path.numpy(), "rb"))
#         rgb = rgbd[0]  # shape: (240, 180, 3)
#         depthmap = rgbd[1]  # shape: (240, 180)

#         rgb = preprocess_depthmap(rgb)
#         rgb = rgb / 255.

#         depthmap = preprocess_depthmap(depthmap)
#         depthmap = depthmap / max_value
#         depthmap = tf.expand_dims(depthmap, -1)    # shape: (240, 180, 1)

#         rgbd = tf.concat([rgb, depthmap], axis=2)
#         rgbd = tf.image.resize(rgbd, (CONFIG.IMAGE_TARGET_HEIGHT, CONFIG.IMAGE_TARGET_WIDTH))
#         targets = preprocess_targets(targets, CONFIG.TARGET_INDEXES)
#         return rgbd, targets

#     rgbd, targets = tf.py_function(py_load_pickle, [path, max_value], [tf.float32, tf.float32])
#     rgbd.set_shape((CONFIG.IMAGE_TARGET_HEIGHT, CONFIG.IMAGE_TARGET_WIDTH, 4))
#     targets.set_shape((len(CONFIG.TARGET_INDEXES,)))
#     return rgbd, targets

def tf_load_pickle(path, max_value):
    def py_load_pickle(path, max_value):
        rgbd, targets = pickle.load(open(path.numpy(), "rb"))
        rgb = rgbd[0]  # shape: (240, 180, 3)
        depthmap = rgbd[1]  # shape: (240, 180)

        rgb = preprocess_depthmap(rgb)
        rgb = rgb / 255.

        depthmap = preprocess_depthmap(depthmap)
        depthmap = depthmap / max_value
        depthmap = tf.expand_dims(depthmap, -1)  # shape: (240, 180, 1)
        rgbd = tf.concat([rgb, depthmap], axis=2)
        rgbd = tf.image.resize(rgbd, (CONFIG.IMAGE_TARGET_HEIGHT, CONFIG.IMAGE_TARGET_WIDTH))
        targets = preprocess_targets(targets, CONFIG.TARGET_INDEXES)
        return rgbd, targets

    rgbd, targets = tf.py_function(py_load_pickle, [path, max_value], [tf.float32, tf.float32])
    rgbd.set_shape((CONFIG.IMAGE_TARGET_HEIGHT, CONFIG.IMAGE_TARGET_WIDTH, NUM_INPUT_CHANNELS))
    targets.set_shape((len(CONFIG.TARGET_INDEXES,)))
    return rgbd, targets

In [ ]:
# Create dataset for training.
paths = paths_training
dataset = tf.data.Dataset.from_tensor_slices(paths)
dataset_norm = dataset.map(lambda path: tf_load_pickle(path, CONFIG.NORMALIZATION_VALUE))
dataset_norm = dataset_norm.cache()
dataset_norm = dataset_norm.prefetch(tf.data.experimental.AUTOTUNE)
dataset_norm = dataset_norm.shuffle(CONFIG.SHUFFLE_BUFFER_SIZE)
dataset_training = dataset_norm
del dataset_norm
# Note: Now the datasets are prepared.

# Create the model.
input_shape = (CONFIG.IMAGE_TARGET_HEIGHT, CONFIG.IMAGE_TARGET_WIDTH, NUM_INPUT_CHANNELS)
model = create_cnn(input_shape, dropout=False)

In [ ]:
model.compile(optimizer='adam', loss="mse", metrics=["mae"])

# LR Find

In [ ]:
lr_finder = LRFinder(start_lr=1e-4)
_ = model.fit(dataset_training.batch(CONFIG.BATCH_SIZE), epochs=20, callbacks=[lr_finder], verbose=2)
lr_finder.plot()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
IGNORE = 40
# aaa = list(zip(lr_finder.lrs[:IGNORE], lr_finder.losses[:IGNORE]))
fig, ax = plt.subplots(1, 1)
ax.set_ylabel('Loss')
ax.set_xlabel('Learning Rate (log scale)')
ax.set_xscale('log')
ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%.0e'))
ax.plot(lr_finder.lrs[:-IGNORE], lr_finder.losses[:-IGNORE]);

In [ ]:
plt.cla()

In [ ]:
lr_finder = LRFinder(end_lr=3) #0.02290) # otherwise plot looks unusable 
_ = model.fit(dataset_training.batch(CONFIG.BATCH_SIZE), epochs=5, callbacks=[lr_finder], verbose=2)
lr_finder.plot()

In [ ]:
lr_finder = LRFinder(end_lr=0.3) #0.02290) # otherwise plot looks unusable 
_ = model.fit(dataset_training.batch(CONFIG.BATCH_SIZE), epochs=5, callbacks=[lr_finder], verbose=2)
lr_finder.plot()

In [ ]:
# LRFinder??

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(lr_finder.losses)
axes = plt.gca()
# axes.set_xlim([xmin,xmax])
axes.set_ylim([4000,9000])
plt.show()

In [ ]:
lr_finder.lrs[63]

In [ ]:
list(zip(lr_finder.lrs, lr_finder.losses))

In [ ]:
len(dataset_training)

In [ ]:
model.summary()